# Lets create our environment together

## Create the containers


In [ ]:
docker compose -f ".devcontainer/docker-compose.yml" up -d

Container dbachecks1  Creating
Container dbachecks2  Creating
Container dbachecks2  Created
Container dbachecks1  Created
Container devcontainer-app-1  Creating
Container devcontainer-app-1  Created
Container dbachecks1  Starting
Container dbachecks2  Starting
Container dbachecks1  Started
Container devcontainer-app-1  Starting
Container dbachecks2  Started
Container devcontainer-app-1  Started


## remove any old files

In [ ]:
Get-ChildItem C:\temp\xlsx | Remove-Item -Force

# Safety First

We must always make sure that we are safe and sound


In [ ]:
Get-SecretStoreConfiguration

If we dont have a vault already in place lets create one


In [ ]:
Register-SecretVault -Name BeardSecrets -ModuleName Microsoft.PowerShell.SecretStore -DefaultVault

and create a couple of secrets for the rest of the code

In [ ]:
Set-Secret -Name sqladmin -Secret “sqladmin”
Set-Secret -Name sqladminpwd -Secret "dbatools.IO"

# copy over the backup file

In [ ]:
$dockerlocation = '\\wsl.localhost\docker-desktop-data\version-pack-data\community\docker\volumes\powershellwithexcel_devcontainer_mydata\_data'
$dockerlocation = '\\wsl.localhost\docker-desktop-data\version-pack-data\community\docker\volumes\devcontainer_mydata\_data'
$Adbackuplocation = 'D:\SQLBackups\AdventureWorks2017.bak'

Copy-Item $Adbackuplocation $dockerlocation

## Set some values

In [ ]:
## first we will set some values that will be used throughout the notebook
$dbachecks1 = 'localhost,7401' # unless we are inside the container Rob
$dbachecks2 = 'localhost,7402' # unless we are inside the container Rob
$benscreds = New-Object System.Management.Automation.PSCredential ((Get-Secret -Name sqladmin -AsPlainText), (Get-Secret -Name sqladminpwd))


$PSDefaultParameterValues = @{
    "*dba*:SqlInstance" = $dbachecks1 
    "*dba*:SqlCredential" = $benscreds 
}

$sql

## Restore a database

In [ ]:
Restore-DbaDatabase  -Path /var/opt/backups/AdventureWorks2017.bak -DatabaseName AdventureWorks2017 -WithReplace


ComputerName         : localhost
InstanceName         : MSSQLSERVER
SqlInstance          : mssql1
BackupFile           : /var/opt/backups/AdventureWorks2017.bak
BackupFilesCount     : 1
BackupSize           : 206.08 MB
CompressedBackupSize : 47.96 MB
Database             : AdventureWorks2017
Owner                : sqladmin
DatabaseRestoreTime  : 00:00:02
FileRestoreTime      : 00:00:02
NoRecovery           : False
RestoreComplete      : True
RestoredFile         : AdventureWorks2017_log.ldf,AdventureWorks2017.mdf
RestoredFilesCount   : 2
Script               : {RESTORE DATABASE [AdventureWorks2017] FROM  DISK = 
                       N'/var/opt/backups/AdventureWorks2017.bak' WITH  FILE = 1,  MOVE 
                       N'AdventureWorks2017' TO N'/var/opt/mssql/data/AdventureWorks2017.mdf',  
                       MOVE N'AdventureWorks2017_log' TO 
                       N'/var/opt/mssql/data/AdventureWorks2017_log.ldf',  NOUNLOAD,  REPLACE,  
                       STATS = 10}
Res

## Create some Logins and Users

In [ ]:
$logins =  'benji', 'bweissman','akamman', 'alevy', 'beardapp1', 'beardapp2', 'beardapp3', 'beardapp4', 'beardapp5', 'beardapp6', 'beardapp7', 'beardapp8', 'clemaire', 'csilva', 'fatherjack', 'gsartori', 'jamrtin', 'Reporting1', 'Reporting2', 'Reporting3', 'Reporting4', 'smelton', 'SockFactoryApp_User', 'soneill',  'Support1', 'Support2', 'Support3', 'Support4', 'Support5', 'Support6', 'tboggiano', 'thebeard', 'wdurkin','Argenis_Fernandez','MrsFatherJack','Cathrine_Wilhelmsen','Rie_Irish','Monica_Rathbun','Benji_BenWeissman','Joey_Dantoni', 'John_Morehouse', 'Bob_Facts_Pusatari','Containers_Nocentino','Andrew_Pruski','Matt_Gordon','Thomas_Grohser','Dirk_Hondong','Father_Johan_Brattas','Drew_FurggerwellyIgiveuptryingtospellit'

$secpwd = (Get-Secret -Name sqladminpwd)
foreach ($login in $logins) {
    New-DbaLogin -Login $login -SecurePassword $secpwd -Force | Out-Null
}

foreach ($login in $logins) {
        New-DbaDbUser   -Database AdventureWorks2017 -Login $login  | Out-Null
        Add-DbaDbRoleMember   -Database AdventureWorks2017 -Role db_owner -User $login -Confirm:$false | Out-Null
    
}

Repair-DbaDbOrphanUser  -Database AdventureWorks2017